In [5]:
import dspy
lm = dspy.LM('ollama_chat/llama3.2', api_base='http://localhost:11434')
dspy.configure(lm=lm)

# lm.history[-1].keys()  # access the last call to the LM, with all metadata



IndexError: list index out of range

In [4]:
len(lm.history) 

1

In [ ]:
sentence = "the power of pettiness"  # example from the SST-2 dataset.

classify = dspy.Predict('sentence -> sentiment: bool')  # we'll see an example with Literal[] later
classify(sentence=sentence).sentiment

print(f"Reasoning: {response.reasoning}")
print(f"Answer: {response.answer}")

# question = "How do the rhizome and lines of flight relate to Deleuze?"
# rag(context=search_wikipedia(question), question=question)

False

In [1]:
import pandas as pd
from typing import List, Union

class ColBERTv2:
    def __init__(self, csv_path: str):
        # Load the CSV into a DataFrame
        self.data = pd.read_csv(csv_path)
        # Ensure there is a column named "Ideas" to search
        if "Ideas" not in self.data.columns:
            raise ValueError("CSV must contain a column named 'Ideas'.")

    def __call__(self, query: str, k: int = 10, simplify: bool = False) -> Union[List[str], List[dict]]:
        # Perform a simple search through the "Ideas" column
        results = []
        for idea in self.data["Ideas"]:
            if query.lower() in idea.lower():  # Basic substring matching
                results.append(idea)
                if len(results) >= k:
                    break

        if simplify:
            return results  # List of strings
        else:
            return [{"text": idea} for idea in results]  # List of dictionaries

# Usage Example
csv_path = "151_ideas_updated2.csv"  # Path to your CSV file

# Initialize the ColBERTv2 class with the CSV file
colbert = ColBERTv2(csv_path)

# Example query
query = "what is expression?"
results = colbert(query, k=3, simplify=True)

# Display results
for result in results:
    print("Result:", result)


In [12]:
sentence = "maximize the beauty."  # example from the SST-2 dataset.

classify = dspy.Predict('sentence -> sentiment: bool')  # we'll see an example with Literal[] later
classify(sentence=sentence).sentiment

# 3) Access the output.
print(response.sentiment)

# question = "How do you maximize the beauty?"
# rag(context=search_wikipedia(question), question=question)

NameError: name 'response' is not defined

In [13]:
import pandas as pd
import dspy
from typing import List

# Custom ColBERTv2 class for CSV-based search
class ColBERTv2:
    def __init__(self, csv_path: str):
        self.data = pd.read_csv(csv_path)
        if "Ideas" not in self.data.columns:
            raise ValueError("CSV must contain a column named 'Ideas'.")

    def __call__(self, query: str, k: int = 10, simplify: bool = False) -> List[dict]:
        # Simple substring match search
        results = []
        for idea in self.data["Ideas"]:
            if query.lower() in idea.lower():
                results.append({"text": idea})
                if len(results) >= k:
                    break
        return results if not simplify else [x["text"] for x in results]

# Initialize the ColBERTv2 instance with your CSV file
colbertv2_csv = ColBERTv2(csv_path="151_ideas_updated2.csv")


# Adapted search function
def search_csv(query: str) -> List[str]:
    results = colbertv2_csv(query, k=3, simplify=True)
    return results

# DSPy ChainOfThought setup
rag = dspy.ChainOfThought("context, question -> response")

# Example question
question = "what is the power of pettiness?"

# Pass context from the CSV search to the ChainOfThought model
response = rag(context=search_csv(question), question=question)

# Output response
print(response)


Prediction(
    reasoning='Pettiness can be defined as behavior that is petty, mean-spirited, or vindictive. It often involves taking small, insignificant actions to cause annoyance or harm to others.',
    response="The power of pettiness lies in its ability to create a toxic environment and undermine relationships. When individuals engage in petty behavior, it can escalate quickly, causing more harm than intended. Pettiness can also be a coping mechanism for people dealing with their own emotions, such as anger or frustration. However, it's essential to recognize that pettiness is not an effective way to resolve conflicts or address issues."
)


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import dspy
import logging
from typing import List, Union

lm = dspy.LM('ollama_chat/llama3.2', api_base='http://localhost:11434')
dspy.configure(lm=lm)
app = Flask(__name__)
CORS(app)
logging.basicConfig(level=logging.INFO)

class ColBERTv2:
    def __init__(self, csv_path: str):
        try:
            self.data = pd.read_csv(csv_path)
            if "Ideas" not in self.data.columns:
                raise ValueError("CSV must contain a column named 'Ideas'.")
        except FileNotFoundError:
            logging.error(f"CSV file not found at path: {csv_path}")
            raise

    def __call__(self, query: str, k: int = 3, simplify: bool = True) -> Union[List[str], List[dict]]:
        try:
            if not isinstance(query, str):
                raise TypeError("Query must be a string")
                
            results = []
            for idea in self.data["Ideas"]:
                if query.lower() in str(idea).lower():
                    results.append({"text": str(idea)})
                    if len(results) >= k:
                        break
            return [x["text"] for x in results] if simplify else results
        except Exception as e:
            logging.error(f"Search error: {str(e)}")
            return []

colbertv2_csv = ColBERTv2(csv_path="151_ideas_updated2.csv")
rag = dspy.ChainOfThought("context, question -> response")

@app.route('/chat', methods=['POST'])
def chat():
    try:
        data = request.get_json()
        if not data or 'question' not in data:
            return jsonify({"response": "Invalid request format", "context": []}), 400
            
        question = str(data['question']).strip()
        if not question:
            return jsonify({"response": "Empty question provided", "context": []}), 400

        context = colbertv2_csv(question, k=3, simplify=True)
        response = rag(context=context, question=question)
        
        return jsonify({
            "response": str(response.response) if response else "No response generated",
            "context": context if isinstance(context, list) else []
        })
        
    except Exception as e:
        logging.error(f"API error: {str(e)}")
        return jsonify({
            "response": "Error processing request",
            "context": []
        }), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.202:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.202:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
127.0.0.1 - - [31/Jan/2025 20:07:05] "OPTIONS /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Jan/2025 20:07:05] "OPTIONS /chat HTTP/1.1" 200 -
20:07:05 - LiteLLM:INFO: utils.py:2749 - 
LiteLLM completion() model= llama3.2; provider = ollama_chat
INFO:LiteLLM:
LiteLLM completion() model= llama3.2; provider = ollama_chat
20:07:27 - LiteLLM:INFO: utils.py:944 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
127.0.0.1 - - [31/Jan/2025 20:07:27] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Jan/202

In [19]:
import pandas as pd
import dspy
from typing import List

# Enhanced ColBERTv2 class for CSV-based search
class ColBERTv2:
    def __init__(self, csv_path: str):
        self.data = pd.read_csv(csv_path)
        if "Ideas" not in self.data.columns:
            raise ValueError("CSV must contain a column named 'Ideas'.")

    def __call__(self, query: str, k: int = 10, simplify: bool = False) -> List[dict]:
        # Perform substring match search
        results = []
        for idea in self.data["Ideas"]:
            if query.lower() in idea.lower():  # Basic match
                results.append({"text": idea})
                if len(results) >= k:
                    break
        return results if not simplify else [x["text"] for x in results]

# Initialize the ColBERTv2 instance with your CSV file
colbertv2_csv = ColBERTv2(csv_path="151_ideas_updated2.csv")

# Adapted search function
def search_csv(query: str) -> List[str]:
    results = colbertv2_csv(query, k=3, simplify=True)
    if not results:  # If no matches found, include a default context
        results = ["No exact matches found in the CSV, but here are some ideas to consider."]
    return results

# Correct DSPy ChainOfThought setup
rag = dspy.ChainOfThought("context, question -> response")

# Example question
question = "How do you maximize the beauty?"

# Pass context from the CSV search to the ChainOfThought model
context = search_csv(question)
response = rag(context=context, question=question)
trainset = [dspy.Example(report="LONG REPORT 1", summary="short summary 1"), ...]
article_summary = dspy.Example(article= "This is an article.", summary= "This is a summary.").with_inputs("article")

input_key_only = article_summary.inputs()
non_input_key_only = article_summary.labels()

print("Example object with Input fields only:", input_key_only)
print("Example object with Non-Input fields only:", non_input_key_only)
# Output response
print("Context:", context)
print("Response:", response)


Example object with Input fields only: Example({'article': 'This is an article.'}) (input_keys={'article'})
Example object with Non-Input fields only: Example({'summary': 'This is a summary.'}) (input_keys=None)
Context: ['No exact matches found in the CSV, but here are some ideas to consider.']
Response: Prediction(
    reasoning='The concept of maximizing beauty is subjective and can vary greatly depending on personal taste, cultural background, and individual experiences. Beauty can be found in nature, art, architecture, or even everyday moments.',
    response="While there's no one-size-fits-all approach to maximizing beauty, here are a few ideas to consider: appreciating the small things in life, finding joy in creative pursuits, cultivating meaningful relationships, and embracing diversity in all its forms. Ultimately, beauty is a personal and ever-evolving experience that can be discovered and nurtured with an open heart and mind."
)


In [ ]:
def validate_answer(example, pred, trace=None):
    return example.answer.lower() == pred.answer.lower()

def validate_context_and_answer(example, pred, trace=None):
    # check the gold label and the predicted answer are the same
    answer_match = example.answer.lower() == pred.answer.lower()

    # check the predicted answer comes from one of the retrieved contexts
    context_match = any((pred.answer.lower() in c) for c in pred.context)

    if trace is None: # if we're doing evaluation or optimization
        return (answer_match + context_match) / 2.0
    else: # if we're doing bootstrapping, i.e. self-generating good demonstrations of each step
        return answer_match and context_match
        

NameError: name 'example' is not defined

In [ ]:
import pandas as pd
import dspy
from typing import List, Dict

# Step 1: Create the basic CSV search class
class ColBERTv2:
    def __init__(self, csv_path: str):
        self.data = pd.read_csv(csv_path)
        if "Ideas" not in self.data.columns:
            raise ValueError("CSV must contain a column named 'Ideas'.")

    def __call__(self, query: str, k: int = 10) -> List[str]:
        results = []
        for idea in self.data["Ideas"].values:
            if query.lower() in str(idea).lower():
                results.append(idea)
                if len(results) >= k:
                    break
        return results

# Step 2: Initialize search and define search function
colbertv2_csv = ColBERTv2(csv_path="151_ideas_updated2.csv")

def search(query: str, k: int = 3) -> List[str]:
    return colbertv2_csv(query, k=k)

# Step 3: Define RAG module with correct signature format
class RAG(dspy.Module):
    def __init__(self, num_docs=3):
        super().__init__()
        self.num_docs = num_docs
        
        # Correct signature format: inputs -> outputs
        self.retrieve_and_respond = dspy.ChainOfThought(
            "context, question -> response"
        )

    def forward(self, question):
        # Retrieve context
        context = search(question, k=self.num_docs)
        
        # Generate response
        prediction = self.retrieve_and_respond(
            context=context,
            question=question
        )
        
        return prediction.response

# Step 4: Create simple training examples
def generate_training_examples() -> List[Dict]:
    # Start with just a few simple examples
    return [
        {"question": "How can we maximize beauty?"},
        {"question": "What are some creative ideas?"}
    ]

# Step 5: Set up optimization function
def optimize_rag():
    # Create training set
    trainset = generate_training_examples()
    
    # Initialize the optimizer with basic settings
    tp = dspy.MIPROv2(
        metric=dspy.evaluate.SemanticF1(decompositional=True),
        auto="medium",
        num_threads=4  # Reduced from 24 for testing
    )
    
    # Compile and optimize
    optimized_rag = tp.compile(
        RAG(),
        trainset=trainset,
        max_bootstrapped_demos=2,
        max_labeled_demos=2
    )
    
    return optimized_rag

# Step 6: Main function with error handling
def main():
    try:
        print("Optimizing RAG system...")
        optimized_rag = optimize_rag()
        
        question = "How do you maximize the beauty?"
        print(f"\nQuestion: {question}")
        
        response = optimized_rag(question)
        print(f"\nResponse: {response}")
        
    except Exception as e:
        
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

NameError: name 'generate_answer' is not defined

In [30]:
import pandas as pd
import dspy
from typing import List, Dict
import re

# Step 1: CSV handling class
class ColBERTv2:
    def __init__(self, csv_path: str):
        self.data = pd.read_csv(csv_path, quoting=1)
        if "Ideas" not in self.data.columns:
            raise ValueError("CSV must contain a column named 'Ideas'.")
        
        self.processed_ideas = self._preprocess_ideas()
    
    def _preprocess_ideas(self) -> List[Dict]:
        processed = []
        for idea in self.data["Ideas"].values:
            text = re.sub(r'^\d+\)\s*', '', str(idea))
            parts = text.split('-', 1)
            if len(parts) > 1:
                title = parts[0].strip()
                content = parts[1].strip()
            else:
                title = ""
                content = parts[0].strip()
            
            processed.append({
                "title": title,
                "content": content,
                "full_text": text
            })
        return processed

    def __call__(self, query: str, k: int = 10) -> List[str]:
        results = []
        query = query.lower()
        
        for idea in self.processed_ideas:
            if (query in idea["title"].lower() or 
                query in idea["content"].lower()):
                results.append(idea["full_text"])
                if len(results) >= k:
                    break
        return results

# Step 2: Initialize search
colbertv2_csv = ColBERTv2(csv_path="151_ideas_updated2.csv")

def search(query: str, k: int = 3) -> List[str]:
    results = colbertv2_csv(query, k=k)
    return [f"Relevant idea: {result}" for result in results]

# Step 3: RAG module with fixed signature
class RAG(dspy.Module):
    def __init__(self, num_docs=3):
        super().__init__()
        self.num_docs = num_docs
        # Simplified signature without additional instructions
        self.retrieve_and_respond = dspy.ChainOfThought("context, question -> response")

    def forward(self, question):
        context = search(question, k=self.num_docs)
        prediction = self.retrieve_and_respond(
            context="\n".join(context),
            question=question
        )
        return prediction.response

# Step 4: Training examples
def generate_training_examples() -> List[Dict]:
    return [
        {
            "question": "How can we maximize beauty?",
            "response": "Based on the philosophical ideas presented, maximizing beauty involves fully channeling the beauty within and actively seeking beauty in every situation. This includes disciplining our senses to focus on beauty in our surroundings, such as appreciating flowers and birds. It's considered a fundamental virtue that requires conscious implementation and can be enhanced by questioning what makes each moment beautiful."
        },
        {
            "question": "What is full expression?",
            "response": "Full expression is about understanding oneself when comfortable and learning to channel authentic self-expression. It recognizes that people face barriers like shyness and distraction that can prevent them from fully expressing themselves. The concept emphasizes the effort required to overcome these obstacles and achieve genuine self-expression."
        },
        {
            "question": "What does expect rising mean?",
            "response": "Expect Rising refers to the concept of constantly increasing expectations, similar to the saying 'give them an inch, they'll take a mile.' It relates to the law of diminishing returns, where adding more of something eventually yields lower incremental benefits. This principle applies to various aspects of life, from material goods to experiences."
        }
    ]

# Step 5: Optimization
def optimize_rag():
    trainset = generate_training_examples()
    
    tp = dspy.MIPROv2(
        metric=dspy.evaluate.SemanticF1(decompositional=True),
        auto="medium",
        num_threads=4
    )
    
    optimized_rag = tp.compile(
        RAG(),
        trainset=trainset,
        max_bootstrapped_demos=3,
        max_labeled_demos=3
    )
    
    return optimized_rag

# Step 6: Main function
def main():
    try:
        print("Optimizing RAG system...")
        optimized_rag = optimize_rag()
        
        test_questions = [
            "How do you maximize the beauty?",
            "What is the power of pettiness?",
            "How can we achieve full expression?"
        ]
        
        for question in test_questions:
            print(f"\nQuestion: {question}")
            response = optimized_rag(question)
            print(f"Response: {response}\n")
            print("-" * 50)
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    main()

2025/01/14 17:47:32 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: False
num_candidates: 19
valset size: 2



Optimizing RAG system...


2025/01/14 17:47:37 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/14 17:47:37 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/14 17:47:37 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


  0%|          | 0/1 [00:00<?, ?it/s]2025/01/14 17:47:37 ERROR dspy.teleprompt.bootstrap: Failed to run or to evaluate example {'question': 'How can we maximize beauty?', 'response': "Based on the philosophical ideas presented, maximizing beauty involves fully channeling the beauty within and actively seeking beauty in every situation. This includes disciplining our senses to focus on beauty in our surroundings, such as appreciating flowers and birds. It's considered a fundamental virtue that requires conscious implementation and can be enhanced by questioning what makes each moment beautiful."} with module = Predict(StringSignature(question, ground_truth, system_response -> reasoning, ground_truth_key_ideas, system_response_key_ideas, discussion, recall, precision
    instructions="Compare a system's response to the ground truth to compute recall and precision of key ideas.\nYou will first enumerate key ideas in each response, discuss their overlap, and then report recall and precisio

...


Average Metric: 0.00 / 0 (0%): 100%|██████████| 2/2 [00:00<00:00, 4415.06it/s]

2025/01/14 17:49:12 INFO dspy.evaluate.evaluate: Average Metric: 0.0 / 2 (0.0%)
2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 0.0 with parameters ['Predictor 0: Instruction 18', 'Predictor 0: Few-Shot Set 8'].
2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 0.0
2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 25 / 25 =====
2025/01/14 17:49:12 ERROR dspy.utils.parallelizer: Error processing item {'question': 'What does expect rising mean?', 'response': "Expect Rising refers to the concept of constantly increasing expectations, similar to the saying 'give them an inch, they'll take a mile.' It relates to the law of diminishin


Average Metric: 0.00 / 0 (0%): 100%|██████████| 2/2 [00:00<00:00, 1926.64it/s]

2025/01/14 17:49:12 INFO dspy.evaluate.evaluate: Average Metric: 0.0 / 2 (0.0%)
2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 0.0 with parameters ['Predictor 0: Instruction 7', 'Predictor 0: Few-Shot Set 18'].
2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 0.0
2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/01/14 17:49:12 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 0.0!




Question: How do you maximize the beauty?
Response: The concept of beauty is subjective and can be influenced by various elements, including historical context, cultural norms, and individual experiences. Maximizing the beauty in a given situation or work of art may require a deep understanding of its intended message, audience, and the emotions it aims to evoke.

--------------------------------------------------

Question: What is the power of pettiness?
Response: Pettiness can be a powerful tool for social manipulation, allowing individuals to exert influence over others without directly confronting them. However, it can also lead to feelings of resentment and frustration among those on the receiving end, ultimately undermining any potential power or control that pettiness may offer.

--------------------------------------------------

Question: How can we achieve full expression?
Response: To achieve full expression in a given statement or condition, it is essential to carefully 

In [25]:
# failed advanced rag
import pandas as pd
import dspy
from typing import List

# Custom ColBERTv2 class for CSV-based search
class ColBERTv2:
    def __init__(self, csv_path: str):
        self.data = pd.read_csv(csv_path)
        if "Ideas" not in self.data.columns:
            raise ValueError("CSV must contain a column named 'Ideas'.")

    def __call__(self, query: str, k: int = 10, simplify: bool = False) -> List[dict]:
        # Simple substring match search
        results = []
        for idea in self.data["Ideas"]:
            if query.lower() in idea.lower():
                results.append({"text": idea})
                if len(results) >= k:
                    break
        return results if not simplify else [x["text"] for x in results]

# Initialize the ColBERTv2 instance with your CSV file
colbertv2_csv = ColBERTv2(csv_path="151_ideas_updated2.csv")

# Adapted search function
def search_csv(query: str) -> List[str]:
    results = colbertv2_csv(query, k=3, simplify=True)
    return results

# DSPy ChainOfThought setup with explicit CSV dependency
rag = dspy.ChainOfThought(
    signature=(
        "context, question -> response\n"
        "context: A list of ideas directly retrieved from the CSV based on the user's query.\n"
        "Only use these ideas to answer the question. If no matches are found, respond with\n"
        "'No relevant information available in the CSV.'\n"
        "question: The user's query.\n"
        "response: A response strictly based on the ideas provided in the context."
    )
)

# Example question
question = "How do you maximize the beauty?"

# Pass context from the CSV search to the ChainOfThought model
response = rag(context=search_csv(question), question=question)

# Output response
print(response)

SyntaxError: unterminated string literal (detected at line 2) (<unknown>, line 2)

In [ ]:
math = dspy.ChainOfThought("question -> answer: float")
math(question="Two dice are tossed. What is the probability that the sum equals two?")